# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Цель проекта - изучить влияние таких факторов как семейное положение, доход, количество детей заемщика  на способность потенциального вернуть кредит банку.
## План работы
Для выполнения проекта нужно решить следующие задачи:

1. Изучить информацию о таблице
2. Обработка пропусков, определить дубликаты и удалить их
3. Лемматизация столбца с целями кредита
4. Категоризация данных
5. Ответить на  следующие 
    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?
6. Сделать выводы  


 Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [4]:
data.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [5]:
data.loc[:, 'income_type']

0        сотрудник
1        сотрудник
2        сотрудник
3        сотрудник
4        пенсионер
           ...    
21520    компаньон
21521    пенсионер
21522    сотрудник
21523    сотрудник
21524    сотрудник
Name: income_type, Length: 21525, dtype: object

## Вывод

Рассмотрим подробнее предоставленную таблицу. В ней 12 столбцовб 21525 строк; тип данных у каждого столбца следующий:  
 
 
| Column            |  Type          | 
| ------------------| :-------------:| 
| children          |  int64         | 
| days_employed     |  float64       |   
| dob_years         |  int64         |   
| education         |  object        | 
| education_id      |  int64         |   
| family_status     |  object        |   
| family_status_id  |  int64         | 
| gender            |  object        |   
| income_type       |  object        |  
| debt              |  int64         | 
| total_income      |  float64       |   
| purpose           |  object        |


   Подробно разберём, какие в df столбцы и какую информацию они содержат:
   
*children* — количество детей в семье 

*days_employed* — общий трудовой стаж в днях

*dob_years* — возраст клиента в годах

*education* — уровень образования клиента 

*education_id* — идентификатор уровня образования 

*family_status* — семейное положение 

*family_status_id* — идентификатор семейного положения 

*gender* — пол клиента 

*income_type* — тип занятости

*debt* — имел ли задолженность по возврату кредитов 

*total_income* — ежемесячный доход 

*purpose* — цель получения кредита

   Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные  значения. Каждая строка таблицы содержит информацию по потенциального заёмщику : возрасте, семейном положении, уровне образования, дохода и т.д.
   Явные проблемы с данными: пропуски, разный регистр, есть отрицательные значения, нет единого названия целей, излишние данные( доход с точностью до 6 знаком после запятой = это лишнее).

**Все переменные таблицы можно разделить на три группы:**

    -логические(gender, debt)  
    
    -категориальные(education,iducation_id, family_status_id, family_status, purpose, gender, income_type)    
    
    -количественные(total_income, days_employed,dob_years, children)
   
    

# Предобработка данных

Поищем артефакты в данных

In [6]:
#Найдем  все уникальные значения в столбце
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [7]:
#Вряд ли имели в виду 20 детей, а скорее 2 ( так как нет никого ни с 18, ни с 15). Также заменим одного отрицательного
#ребенка одним положительным
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


 ## Обработка пропусков

In [8]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [9]:
data[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


В некоторых строках таблицы пропущены данные о стаже работы и доходе. Возможно, это связано с тем, что человек работал без официального трудоустройста.

In [43]:

data_pivot = data.pivot_table(values=['total_income', 'days_employed'], index = 'income_type', aggfunc=('count'))
print(data_pivot)

                 days_employed  total_income
income_type                                 
безработный                  2             2
в декрете                    1             1
госслужащий               1457          1457
компаньон                 5078          5078
пенсионер                 3829          3829
предприниматель              2             2
сотрудник                11084         11084
студент                      1             1


d = data.pivot_table(values=['total_income', 'days_employed'], 
                        index = 'income_type', 
                        aggfunc=('count','size'))

In [44]:

d['isNaN_days'] = d['days_employed']['size'] - d['days_employed']['count']
d['isNaN_income'] = d['total_income']['size'] - d['total_income']['count']
d[['isNaN_days', 'isNaN_income']]

,isNaN_days,isNaN_income
,,
income_type,,
безработный,0,0
в декрете,0,0
госслужащий,147,147
компаньон,508,508
пенсионер,413,413
предприниматель,1,1
сотрудник,1105,1105
студент,0,0


Не уверена, что это то что нужно. но вроде сводные таблицы усвоились лучше.
Примерно 10% всех пропусков по группа по типам занятости.


In [13]:
# Заполним пропущенные данные в столбце total_income. Преположим, что доход в большей степени зависит от типа занятости и 
#и заполним пропуски медианным значением дохода для каждого из типов.
data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'), inplace=True)


In [14]:
#Заполним пропущенные значения в столбце days_employed. Предположим, что стаж зависит от возраста заемщика и заполним
#пропуски медианным значением возраста .
data['days_employed'].fillna(data.groupby('dob_years')['days_employed'].transform('median'), inplace =True)


**Думаю, что данные пропущенные намеренно. Человек не работал официально, следовательно не имел стажа и официального дохода**

### Вывод

Пропущенные значения стажа заполнены исходя из медианных значений стажа для возраста заявителя. А более важный столбец с доходом свои пропущенные значения получил в зависимости от занятости заемщика.

## Замена типа данных

In [15]:
# Для начала изменим тип данных в столбце total_income . Нам не нужная такая точность в значениях. Изменим тип данных на int
data['total_income'].astype('int')


0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int64

In [16]:
#Также для удобства работы заменим тип данных в столбце стажа. Также воспользуемся методом .astype('int')
data['days_employed'].astype('int')

0         -8437
1         -4024
2         -5623
3         -4124
4        340266
          ...  
21520     -4529
21521    343937
21522     -2113
21523     -3112
21524     -1984
Name: days_employed, Length: 21525, dtype: int64

### Вывод

С помощью метода `.astype` заменим вещественный тип данных в столбцах стажа и дохода на целочисленный.

## Обработка дубликатов

In [17]:
#Найдем количество дубликатов в строках с типом данных object
print(data['education'].value_counts(), '\n')
print(data['family_status'].value_counts(), '\n')
print(data['gender'].value_counts(), '\n')
print(data['income_type'].value_counts(), '\n')
print(data['purpose'].value_counts(), '\n')

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64 

свадьба                                 

In [18]:
data['education'] = data['education'].str.lower()

In [19]:
#Проверим  как теперь обстоят дела с дубликатами в графе образования
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [20]:
# Графу цель кредита на дубликаты проверять пока не было смысла. Необходимо провести лемматизацию . А пока проверим какие 
# еще остались дубликаты, после приведения к нижнему регистру.
data.duplicated().sum()

71

In [21]:
data = data.drop_duplicates()

In [22]:
data.duplicated().sum()

0

### Вывод

Для начала проверим количество уникальных значений типа object с помощью метода value_counts(). Внимание привлекала колонка education. В таблице было значительное количество дубликатов, связанное с вариантами написания уровня образования. При помощи метода str.lower() мы привели все значения к нижнему регистру. Оставшиеся значения удалили с помощью метода drop_duplicates().

## Лемматизация

In [23]:
# Посмотрим все варианты написания цели кредита
print(data['purpose'].value_counts(), '\n')

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [24]:
from pymystem3 import Mystem
m = Mystem()
def new_category(purpose):
    lem = m.lemmatize(purpose)
    if ('недвижимость' in lem) or ('жилье' in lem):
        return 'Недвижимость'
    elif ('свадьба' in lem) :
        return 'Свадьба'
    elif ('автомобиль' in lem) :
        return 'Автомобиль'
    else:
        return ('Образование')
data['new_category'] = data['purpose'].apply(new_category)
print(data.head(10))


   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   среднее             1   
8         2   -6929.865299         35    высшее             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

### Вывод

Пункт цель кредита был заполнен множеством вариантов. Провели лемматизацию, сгруппировав в 4 группы. Лемматизация была проведена с помощью метода m.lemmatize библиотеки pymystem3. Для применения сразу ко всему столбцу создали функцию new_category(purpose).

## Категоризация данных

In [25]:
#С помощью метода describe() получим некоторую сводную информацию по всей таблице. По умолчанию будет выдана 
#информация только для количественных признаков. Это общее их количество (count), среднее значение (mean), 
#стандартное отклонение (std), минимальное (min), 
#макcимальное (max) значения, медиана (50%) и значения нижнего (25%) и верхнего (75%) квартилей:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04
mean,0.480563,62892.742151,43.271231,0.817097,0.973898,0.081150,1.653201e+05
std,0.756069,140213.652410,12.570822,0.548674,1.421567,0.273072,9.818730e+04
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2523.810846,33.000000,1.000000,0.000000,0.000000,1.076239e+05
50%,0.000000,-1264.995814,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,-318.151112,53.000000,1.000000,1.000000,0.000000,1.958209e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [26]:
#Разделим данные по доходам на 10 групп. Воспользуемся для этого метеодом qcut(). Точность до целого(precision=0)
# Каждому децилю присвоим свой номер.
data['decile'] = pd.qcut(data['total_income'], q=10, labels=['1', '2', '3', '4','5', '6', '7', '8', '9', '10'])
data['income_interval'] = pd.qcut(data['total_income'], q=10, precision = 0)
#data['decile'].value_counts()
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  new_category decile       income_interval  
0               покупка жилья  Недвижимость    

In [27]:
# Создадим отдельный словарь с типами образования, каждой категории будет соответствовать номер.
education_dict = data[['education', 'education_id']]
print(education_dict.head(10))

  education  education_id
0    высшее             0
1   среднее             1
2   среднее             1
3   среднее             1
4   среднее             1
5    высшее             0
6    высшее             0
7   среднее             1
8    высшее             0
9   среднее             1


In [28]:
#В словаре большое количество дубликатов, удалим их. Теперь представление данных о уровне образования стало компактнее
education_dict = education_dict.drop_duplicates().reset_index(drop = True)
print(education_dict.head())

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


In [29]:
#Также поступим с семейным положением. Создадим словарь, удалим дубликаты.
family_status_dict = data[['family_status', 'family_status_id']]
#print(family_status_dict.head(10))
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict.head())

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


### Вывод

Провели категоризацию данных.
Разбили данные об уровне дохода на 10 групп. Каждой из них присвоили цифровое значение.
Семейное положение и уровень образования также разбили на несколько групп. Для этого воспользовались конструкцией "словарь".


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [30]:
#Изучим влияние количества детей в семье на возврат кредита в срок. Для начала поищем аномалии в данных
data['children'].value_counts()


0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [31]:
# Посмотрим , как  влияет количество детей на невозврат кредита. Долг по кредиту заполняется единицей. 
children_count = data.groupby('children')['debt'].count() 
children_sum = data.groupby('children')['debt'].sum()
# Поделим количество должников  в зависимости от количества детей на общее количество семей с таким же семейным составом
proportion_children_debt = children_sum / children_count
print(proportion_children_debt  * 100)

children
0    7.543822
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


### Вывод

Обнаружена явная зависимость между возвратом кредита и количеством детей. Бездетные самые ответственные,  а с ростом количества детей ситуация ухудшается. Процент невозврата по кредиту в семьях с 4 детьми уже 9,75 %. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
# Методом groupby проверим влияет ли семейный статус на возврат кредита. Создадим переменные,в которых будет подсчитываться
#количество заемщиков каждого типа и сколько из них не вернуло кредит. Разделим долников на общее количество по группам и получим п
#процентное соотношение
family_count = data.groupby('family_status')['debt'].count()
family_sum = data.groupby('family_status')['debt'].sum()
proportion_family = family_sum / family_count
print(proportion_family * 100)

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
Name: debt, dtype: float64


### Вывод

Оказалось, что самый низкий процент у вдовцов, а вот холостяки часто не возвращают кредит

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
# Также как и с семейным статусом, поступим с доходом. Найдем процентное соотношение для каждой группы дохода
decile_count = data.groupby('decile')['debt'].count()
decile_sum = data.groupby('decile')['debt'].sum()
proportion_decile = decile_sum / decile_count
print(proportion_decile * 100)

decile
1     7.315937
2     8.717949
3     8.344988
4     8.480895
5     9.066785
6     8.378512
7     8.434296
8     8.391608
9     6.946387
10    7.036347
Name: debt, dtype: float64


In [34]:
income_info = data[['decile', 'income_interval']]
income_info = income_info.drop_duplicates().sort_values(by=['decile']).reset_index(drop=True)
print(income_info)

  decile        income_interval
0      1     (20666.0, 78722.0]
1      2     (78722.0, 98538.0]
2      3    (98538.0, 116009.0]
3      4   (116009.0, 132135.0]
4      5   (132135.0, 142594.0]
5      6   (142594.0, 161336.0]
6      7   (161336.0, 179805.0]
7      8   (179805.0, 214618.0]
8      9   (214618.0, 269826.0]
9     10  (269826.0, 2265604.0]


### Вывод

Самые надежные заемщики это люди с низким доходом , и выше среднего.С доходом  (214618.0, 269826.0] кредиты вовращает большинство.


- Как разные цели кредита влияют на его возврат в срок?

In [35]:
# Посчитаем отношение должников к общему количеству по кадой цели кредита
purpose_count = data.groupby('new_category')['debt'].count()
purpose_sum = data.groupby('new_category')['debt'].sum()
proportion_purpose = purpose_sum / purpose_count
print(proportion_purpose * 100)


new_category
Автомобиль      9.359034
Недвижимость    7.233373
Образование     9.220035
Свадьба         8.003442
Name: debt, dtype: float64


### Вывод

Кредит на недвижимость возвращают лучше всего.

### Шаг 4. Общий вывод

Провели анализ данных о платежеспособности заемщиков.
В данных имелись пропуски, дубликаты и аномалии. Для наибольшего удобства была проведена категоризация данных. Дополнительные категории упростили нам задачу с оценкой перспектив не возврата кредита.
После проведенного анализа мы получили следующие выводы:

-**Дети**  - количество детей серьезно влияет на надежность заемщика. Процент возврата кредита выше всего у клиентов без детей

-**Семейное положение**  - лучше всего возвращают кредит вдовцы, а вот незамужние не отличаются надежностью

-**Цель кредита**  - процент невозврата среди клиентов, взявщих займ на недвижимость, самый низкий

-**Уровень дохода**  - люди с достатком в интеревале от 214618.0 до  269826.0 вовращают кредит лучше всего.
